In [1]:
key_vault_name = 'kv_to-be-replaced'

StatementMeta(, , , Waiting, )

In [2]:
from trident_token_library_wrapper import PyTridentTokenLibrary as tl

def get_secrets_from_kv(kv_name, secret_name):

    access_token = mssparkutils.credentials.getToken("keyvault")
    kv_endpoint = f'https://{kv_name}.vault.azure.net/'
    return(tl.get_secret_with_token(kv_endpoint,secret_name,access_token))

openai_api_type = "azure"
openai_api_version  = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-VERSION")
openai_api_base = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-ENDPOINT")
openai_api_key = get_secrets_from_kv(key_vault_name,"AZURE-OPENAI-KEY")

StatementMeta(, , , Waiting, )

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, LongType, TimestampType
import os

StatementMeta(, , , Waiting, )

In [8]:
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, MapType, LongType, TimestampType
import os

folder_path = 'Files/data/conversation_input/'

# Define the schema for the nested Messages in the Conversation
message_schema = StructType([
    StructField("Id", StringType(), True),
    StructField("ReferenceId", StringType(), True),
    StructField("EventType", StringType(), True),
    StructField("EventTime", StringType(), True),
    StructField("ConversationId", StringType(), True),
    StructField("Value", StringType(), True),
    StructField("UserId", StringType(), True),
    StructField("CustomProperties", MapType(StringType(), StringType()), True)
])

# Define the schema for the Conversation
conversation_schema = StructType([
    StructField("ConversationId", StringType(), True),
    StructField("Messages", ArrayType(message_schema), True),
    StructField("StartTime", TimestampType(), True),
    StructField("EndTime", TimestampType(), True),
    StructField("Merged_content", StringType(), True),
    StructField("Merged_content_user", StringType(), True),
    StructField("Merged_content_agent", StringType(), True),
    StructField("Full_conversation", StringType(), True),
    StructField("Duration", LongType(), True)  # New field for duration
])

# Define the complete schema for the JSON document
schema = StructType([
    StructField("AgentName", StringType(), True),
    StructField("AgentId", StringType(), True),
    StructField("Team", StringType(), True),
    StructField("ResolutionStatus", StringType(), True),
    StructField("CallReason", StringType(), True),
    StructField("CallerID", StringType(), True),
    StructField("Conversation", conversation_schema, True)
])

df = None
df = spark.read.option("multiLine", True).schema(schema).option("mode", "FAILFAST").json(folder_path)

# Print the schema to confirm it matches the intended design
# df.printSchema()

#use the legacy time parser policy
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# Update Duration field with the duration from StartTime to Endtime in milliseconds
df = df.withColumn("Conversation", df["Conversation"].withField("Duration", 
                                                (F.unix_timestamp(df["Conversation"]["EndTime"], 'yyyy-MM-dd\'T\'HH:mm:ss') - 
                                                 F.unix_timestamp(df["Conversation"]["StartTime"], 'yyyy-MM-dd\'T\'HH:mm:ss')) / 60))

df = df.withColumn("Conversation", df["Conversation"].withField("ConversationDate", (df["Conversation"]["StartTime"])))

df.select("Conversation.*").show()
# df = spark.read.option("multiLine", True).schema(schema).json(folder_path)

# # Show the DataFrame to verify the contents
# df.show(truncate=False)

# # To specifically print the ConversationId
# df.select("Conversation.ConversationId").show(truncate=False)

# # Print the schema to confirm it matches the intended design
# df.printSchema()


StatementMeta(, , , Waiting, )

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|      ConversationId|            Messages|           StartTime|             EndTime|      Merged_content| Merged_content_user|Merged_content_agent|   Full_conversation|          Duration|    ConversationDate|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+
|08800856-3e8c-403...|[{d6bd8606-68b1-4...|2024-03-19 07:51:...|2024-03-19 07:55:...|Thank you for cal...|Hi Nathan, my nam...|Thank you for cal...|Agent: Thank you ...|3.8833333333333333|2024-03-19 07:51:...|
|17eb4abf-6c63-465...|[{fe90e6f4-5ac8-4...|2024-02-13 01:33:...|2024-02-13 01:36:...|Thank you for cal...|Hi Isaac, my name...|Thank you for cal...|Agent: Thank

In [9]:
# Select specific columns, including nested ones
selected_df = df.select(
    "AgentName",
    "AgentId",
    "Team",
    "ResolutionStatus",
    "CallReason",
    "CallerID",
    "Conversation.ConversationId",
    "Conversation.StartTime",
    "Conversation.EndTime",
    "Conversation.ConversationDate",
    "Conversation.Merged_content",
    "Conversation.Merged_content_user",
    "Conversation.Merged_content_agent",
    "Conversation.Full_conversation",
    "Conversation.Duration"
)

StatementMeta(, , , Waiting, )

In [19]:
# table_name = "ckm_conv_processed_raw"  # Replace with your table name

# # Get the schema of the table
# table_schema = spark.table(table_name).schema

# # Compare the schema of DataFrame and the table
# if str(df.schema) == str(table_schema):
#     print("The DataFrame schema matches the table schema.")
# else:
#     print("The DataFrame schema does not match the table schema.")
#     print("\nDataFrame schema:\n" + str(df.schema))
#     print("\nTable schema:\n" + str(table_schema))


StatementMeta(, dbbc1734-102a-4117-aa82-1a58d51aecec, 21, Finished, Available)

In [4]:
# table_name = "ckm_conv_processed_raw"  # Replace with your table name
# spark.sql(f"DROP TABLE IF EXISTS {table_name}")

StatementMeta(, e72efe3e-6ced-487c-86c7-559dbf22094a, 6, Cancelled, Waiting)

In [21]:
#table_name = "ckm_conv_processed"  # Replace with your table name
#spark.sql(f"DROP TABLE IF EXISTS {table_name}")

StatementMeta(, dbbc1734-102a-4117-aa82-1a58d51aecec, 23, Finished, Available)

In [22]:
# selected_df.write.saveAsTable("ckm_conv_processed_raw")


StatementMeta(, dbbc1734-102a-4117-aa82-1a58d51aecec, 24, Finished, Available)

In [1]:

selected_df = spark.sql("SELECT * FROM ckm_conv_processed_raw where summary is NULL or summary == ''")
# display(selected_df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, bac80d3b-b078-42fb-9662-17fd84567771)

In [ ]:
# display(spark.sql("SELECT * FROM ckm_conv_processed_raw where ConversationId = '1396b34a-0a43-447d-9b28-ab5beec4e976'"))

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, f7ac2b57-3543-4eed-b758-3b9c64f94c08)

In [10]:
# # Run when Open ai timesout 
selected_df = spark.sql("SELECT * FROM ckm_conv_processed_raw LIMIT 5")
# display(selected_df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 7968035f-bccb-43ae-a8c7-21a38d064168)

In [11]:
# display(selected_df)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 5d9f1a06-73c8-4b7b-8b8d-e0a4b1921e7b)

In [3]:
import os
import openai
import json
import time
import ast

# Set the OpenAI API key once outside the function
openai.api_version = openai_api_version
openai.api_base = openai_api_base
openai.api_key =  openai_api_key

# Function to get details from a conversation
def get_details(input_text):
    time.sleep(4)

    # openai.api_version = openai_api_version
    # openai.api_base = openai_api_base
    # openai.api_key =  openai_api_key

    # Construct the prompt for the OpenAI API
    prompt = '''You are a JSON formatter for extracting information out of a single chat conversation. 
            Summarize the conversation, key: summary .
            Is the customer satisfied with the agent interaction (Yes or No), key: satisfied . 
            Identify the sentiment of the customer as (Positive, Negative or Neutral),key : avgSentiment . 
            Identify the origin city of travel,key: OriginCity . 
            Identify the destination city of travel,key : DestinationCity . 
            Identify the single primary complaint of the conversation in 6 words or less,key: Complaint . 
            Identify the single primary compliment of the conversation in 6 words or less,key: Compliment . 
            Identify the name of hotel that was mentioned,key: Hotel . 
            Identify the name of airline if mentioned,key: Airline . 
            Identify the name of the agent,key: AgentName .
            Identify the top 10 key phrases as comma seperated string excluding people names , key: keyPhrases .
            Identify the main topic, key: topic .
            Identify the language of the text using ISO 639 two letter language identifier, key: lang .
            Identify input_text translated to english, return the same text if already in english, key: translated_text .
            Answer in JSON machine-readable format, using the keys from above. 
            Pretty print the JSON and make sure that it is properly closed at the end and do not generate any other content.'''

    # Set maximum number of retries
    max_retries = 5
    attempts = 0

    # Loop until maximum retries are reached
    while attempts < max_retries:
        try:
            # Call OpenAI API with the constructed prompt
            response = openai.ChatCompletion.create(
            engine= "gpt-4",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": input_text}
            ],
            response_format={"type": "json_object"})

            # Parse the response from the API
            result = ast.literal_eval(response['choices'][0]['message']['content'])
            # If 'summary' is found in the result, print and return the result
            if 'summary' in result and result['summary'] is not None and result['summary'].strip() != '':
                print(result)
                return result
            else:
                # If 'summary' is not found, increment attempts and try again
                attempts += 1
                print(f"Attempt {attempts} failed. 'summary' not found in result. Trying again.")
                time.sleep(1)
        except Exception as e:
            # If an error occurs, increment attempts and try again
            attempts += 1
            print(f"Attempt {attempts} failed with error: {e}. Trying again.")
            time.sleep(1)

    print("Maximum number of retries reached. Exiting.")
    return {
        'summary': '',
        'satisfied': '',
        'avgSentiment': '',
        'OriginCity': '',
        'DestinationCity': '',
        'Complaint': '',
        'Compliment': "",
        'Hotel': '',
        'Airline': '',
        'AgentName': '',
        'keyPhrases': '',
        'topic': '',
        'lang': '',
        'translated_text': ''
    }


# input_str = '''Thank you for reaching out to the travel agency contact center. My name is Sarah Thompson. How may I assist you today?Hi Sarah, my name is Lisa Johnson. I recently traveled from Chicago to London and I had a terrible experience with the airline and hotel. I'm really frustrated with the service I received.I'm sorry to hear that, Lisa. Can you please provide me with some details about your trip so I can better understand the situation?Sure. I flew with United Airlines from Chicago to London, and I stayed at the Park Plaza Westminster Bridge hotel in London. I encountered issues with both during my trip.I apologize for any inconvenience you experienced. Could you please explain the specific problems you encountered with United Airlines?Absolutely. Firstly, the flight was delayed for more than three hours without any proper explanation. This caused a lot of inconvenience as I had connecting flights and had to reschedule my entire itinerary. Secondly, the onboard service was subpar. The flight attendants seemed disinterested and were not attentive to the passengers' needs.I understand how frustrating these situations can be, Lisa. I apologize for the lack of communication and the inconvenience caused by the delay. Delayed flights can be quite disruptive. Regarding the onboard service, I apologize for the unprofessional behavior of the flight attendants. I will make a note of your concerns and forward them to the airline for review.Thank you, Sarah. I appreciate your understanding. Now, regarding my hotel stay at the Park Plaza Westminster Bridge, the room was not up to standard. It was not properly cleaned, and there were maintenance issues with the bathroom.I apologize for the hotel's shortcomings, Lisa. It can be disappointing when accommodations don't meet expectations. I will contact the hotel management to address the cleanliness and maintenance issues you faced. In the meantime, is there anything specific you would like me to convey to the hotel?I would like them to know that I expect better cleanliness and maintenance in their rooms. It was really disappointing, especially considering the hotel's reputation.I completely understand, Lisa. I will communicate your concerns to the hotel management and emphasize the importance of ensuring a high level of cleanliness and maintenance throughout their property.Thank you, Sarah. I appreciate your assistance. Is there anything else you can do to help resolve these issues?Absolutely, Lisa. To further assist you, I will contact United Airlines to see if they can offer any compensation for the delay and address your concerns about the onboard service. Additionally, I will follow up with the Park Plaza Westminster Bridge hotel to ensure they take appropriate action regarding the cleanliness and maintenance issues in your room. I will keep you updated throughout the process.That sounds good, Sarah. I appreciate your efforts in resolving these matters. Thank you for your assistance.You're most welcome, Lisa. It is our priority to ensure that our customers have a pleasant travel experience. I will work diligently to resolve these issues for you. If you have any other questions or concerns, please don't hesitate to contact me.Thank you, Sarah. I will definitely reach out if I need any further assistance.'''
# res = get_details(input_str)

from pyspark.sql import Row

from pyspark.sql.types import *
from pyspark.sql.functions import *

schema = StructType([
             StructField("summary", StringType(), True),
             StructField("satisfied", StringType(), True),
             StructField("avgSentiment", StringType(), True),
             StructField("OriginCity", StringType(), True),
             StructField("DestinationCity", StringType(), True),
             StructField("Complaint", StringType(), True),
             StructField("Compliment", StringType(), True),
             StructField("Hotel", StringType(), True),
             StructField("Airline", StringType(), True),
             StructField("AgentName", StringType(), True),
             StructField("keyPhrases", StringType(), True),
             StructField("topic", StringType(), True),
             StructField("lang", StringType(), True),
             StructField("translated_text", StringType(), True)
         ])

get_detail_udf = udf(lambda content: get_details(content),returnType=schema)

df_processed = selected_df.select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason",
    "CallerID","Merged_content","Merged_content_agent","Merged_content_user"]) \
                .withColumn("Details", get_detail_udf(col("Merged_content"))) \
                .select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason","CallerID","Merged_content","Merged_content_agent","Merged_content_user", \
                          col("Details.summary").alias("summary"), \
                          col("Details.satisfied").alias("satisfied"), \
                          col("Details.avgSentiment").alias("avgSentiment"), \
                          col("Details.OriginCity").alias("OriginCity"), \
                          col("Details.DestinationCity").alias("DestinationCity"), \
                          col("Details.Complaint").alias("Complaint"), \
                          col("Details.Compliment").alias("Compliment"), \
                          col("Details.Hotel").alias("Hotel"), \
                          col("Details.Airline").alias("Airline"), \
                          # col("Details.AgentName").alias("AgentName"), \
                          col("Details.keyPhrases").alias("keyPhrases"), \
                          col("Details.topic").alias("topic"), \
                          col("Details.lang").alias("lang"), \
                          col("Details.translated_text").alias("translated_text")]) 
# display(df_processed)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, a1425899-4cc5-40e6-b6c0-1a43a39ecbee)

In [7]:
# # # Nalini debugging

# import os
# import openai
# import json
# import time
# import ast

# df = spark.sql("select merged_content from ckm_conv_processed_raw where ConversationId = 'e22d1676-ff5b-4c41-81a5-799cae57bf11'").toPandas()
# # df = spark.sql("select merged_content from ckm_conv_processed_raw where ConversationId = '1396b34a-0a43-447d-9b28-ab5beec4e976'").toPandas()

# for i, row in df.iterrows():
#     # print(row.merged_content)
#     # res1 = get_details(row.merged_content[:100])
#     # print(res1)

#     input_text = row.merged_content #.replace('$','').replace('/','')
#     import re
#     input_text = re.sub('\W+',' ', input_text )

#     openai.api_type = openai_api_type
#     openai.api_version = openai_api_version
#     openai.api_base = openai_api_base
#     openai.api_key =  openai_api_key

#     # Construct the prompt for the OpenAI API
#     prompt = '''You are a JSON formatter for extracting information out of a single chat conversation. 
#             Summarize the conversation, key: summary .
#             Is the customer satisfied with the agent interaction (Yes or No), key: satisfied . 
#             Identify the sentiment of the customer as (Positive, Negative or Neutral),key : avgSentiment . 
#             Identify the origin city of travel,key: OriginCity . 
#             Identify the destination city of travel,key : DestinationCity . 
#             Identify the single primary complaint of the conversation in 6 words or less,key: Complaint . 
#             Identify the single primary compliment of the conversation in 6 words or less,key: Compliment . 
#             Identify the name of hotel that was mentioned,key: Hotel . 
#             Identify the name of airline if mentioned,key: Airline . 
#             Identify the name of the agent,key: AgentName .
#             Identify the top 10 key phrases as comma seperated string excluding people names , key: keyPhrases .
#             Identify the main topic, key: topic .
#             Identify the language of the text using ISO 639 two letter language identifier, key: lang .
#             Identify input_text translated to english, return the same text if already in english, key: translated_text .
#             Answer in JSON machine-readable format, using the keys from above. 
#             Pretty print the JSON and make sure that it is properly closed at the end and do not generate any other content.'''

#     # Set maximum number of retries
#     max_retries = 5
#     attempts = 0
#     response1 = openai.ChatCompletion.create(
#         engine= "gpt-4",
#         messages=[{"role": "system", "content": prompt},{"role": "user", "content": input_text}],
#         response_format={"type": "json_object"})

#     print(response1['choices'][0]['message']['content'])
#     print(ast.literal_eval(response1['choices'][0]['message']['content']))

StatementMeta(, , , Waiting, )

{
  "summary": "John Smith calls Travel Agency for assistance with planning a trip from Atlanta to New York City, including airline and hotel options. Agent Mia Robinson recommends Delta or American Airlines and Marriott Marquis Times Square within his budget. John initially requests more options but eventually decides on Delta and Marriott Marquis. Mia proceeds with booking.",
  "satisfied": "Yes",
  "avgSentiment": "Positive",
  "OriginCity": "Atlanta",
  "DestinationCity": "New York City",
  "Complaint": "",
  "Compliment": "Helpful decision-making assistance",
  "Hotel": "Marriott Marquis",
  "Airline": "Delta, American Airlines",
  "AgentName": "Mia Robinson",
  "keyPhrases": "travel plans, Atlanta to New York City, June 15th to June 20th, Delta, American Airlines, Marriott Marquis, hotel options, airline options, budget, booking",
  "topic": "Travel planning assistance",
  "lang": "en",
  "translated_text": "Thank you for calling Travel Agency this is Mia Robinson How may I assis

In [ ]:
# # Rob debugging

# import os
# import openai
# import json
# import time
# import ast
# from pyspark.sql import Row
# from pyspark.sql.types import *
# from pyspark.sql.functions import *

# # Function to get details from a conversation
# def get_details(input_text):
#     time.sleep(4)

#     openai.api_version = openai_api_version
#     openai.api_base = openai_api_base
#     openai.api_key =  openai_api_key

#     # Construct the prompt for the OpenAI API
#     prompt = '''You are a JSON formatter for extracting information out of a single chat conversation. 
#             Summarize the conversation, key: summary .
#             Is the customer satisfied with the agent interaction (Yes or No), key: satisfied . 
#             Identify the sentiment of the customer as (Positive, Negative or Neutral),key : avgSentiment . 
#             Identify the origin city of travel,key: OriginCity . 
#             Identify the destination city of travel,key : DestinationCity . 
#             Identify the single primary complaint of the conversation in 6 words or less,key: Complaint . 
#             Identify the single primary compliment of the conversation in 6 words or less,key: Compliment . 
#             Identify the name of hotel that was mentioned,key: Hotel . 
#             Identify the name of airline if mentioned,key: Airline . 
#             Identify the name of the agent,key: AgentName .
#             Identify the top 10 key phrases as comma seperated string excluding people names , key: keyPhrases .
#             Identify the main topic, key: topic .
#             Identify the language of the text using ISO 639 two letter language identifier, key: lang .
#             Identify input_text translated to english, return the same text if already in english, key: translated_text .
#             Answer in JSON machine-readable format, using the keys from above. 
#             Pretty print the JSON and make sure that it is properly closed at the end and do not generate any other content.'''

#     # Set maximum number of retries
#     max_retries = 5
#     attempts = 0

#     # Loop until maximum retries are reached
#     while attempts < max_retries:
#         try:
#             # Call OpenAI API with the constructed prompt
#             response = openai.ChatCompletion.create(
#             engine= "gpt-4",
#             messages=[
#                 {"role": "system", "content": prompt},
#                 {"role": "user", "content": input_text}
#             ],
#             response_format={"type": "json_object"})

#             # Parse the response from the API
#             result = ast.literal_eval(response['choices'][0]['message']['content'])
#             # If 'summary' is found in the result, print and return the result
#             if 'summary' in result and result['summary'] is not None and result['summary'].strip() != '':
#                 print(result)
#                 return result
#             else:
#                 # If 'summary' is not found, increment attempts and try again
#                 attempts += 1
#                 print(f"Attempt {attempts} failed. 'summary' not found in result. Trying again.")
#                 time.sleep(1)
#         except Exception as e:
#             # If an error occurs, increment attempts and try again
#             attempts += 1
#             print(f"Attempt {attempts} failed with error: {e}. Trying again.")
#             time.sleep(1)

#     print("Maximum number of retries reached. Exiting.")
#     return {
#         'summary': '',
#         'satisfied': '',
#         'avgSentiment': '',
#         'OriginCity': '',
#         'DestinationCity': '',
#         'Complaint': '',
#         'Compliment': "",
#         'Hotel': '',
#         'Airline': '',
#         'AgentName': '',
#         'keyPhrases': '',
#         'topic': '',
#         'lang': '',
#         'translated_text': ''
#     }

# # Set the OpenAI API key once outside the function
# openai.api_version = openai_api_version
# openai.api_base = openai_api_base
# openai.api_key =  openai_api_key

# # Define the schema
# schema = StructType([
#              StructField("summary", StringType(), True),
#              StructField("satisfied", StringType(), True),
#              StructField("avgSentiment", StringType(), True),
#              StructField("OriginCity", StringType(), True),
#              StructField("DestinationCity", StringType(), True),
#              StructField("Complaint", StringType(), True),
#              StructField("Compliment", StringType(), True),
#              StructField("Hotel", StringType(), True),
#              StructField("Airline", StringType(), True),
#              StructField("AgentName", StringType(), True),
#              StructField("keyPhrases", StringType(), True),
#              StructField("topic", StringType(), True),
#              StructField("lang", StringType(), True),
#              StructField("translated_text", StringType(), True)
#          ])

# # Create a copy of the original DataFrame
# df_processed = selected_df

# # Loop over each row in the DataFrame
# for row in selected_df.rdd.collect():
#     # Get the conversation text
#     input_text = row['Merged_content']

#     # Call the get_details function
#     details = get_details(input_text)

#     # Update the row with the new details
#     for key, value in details.items():
#         df_processed = df_processed.withColumn(key, lit(value))

# # Display the processed DataFrame
# # display(df_processed)



StatementMeta(, , , Waiting, )

{'summary': 'Lisa Smith contacted ABC Travel for assistance in booking a flight from New York City to Los Angeles and a hotel under $150 a night. Agent Aiden provided flight options with Delta, United, and American Airlines, and hotel options at the Hilton Los Angeles Airport and Marriott LAX, detailing their amenities. Lisa chose to book the Delta flight and the Marriott LAX.', 'satisfied': 'Yes', 'avgSentiment': 'Positive', 'OriginCity': 'New York City', 'DestinationCity': 'Los Angeles', 'Complaint': '', 'Compliment': 'Provided personalized options', 'Hotel': 'Marriott LAX, Hilton Los Angeles Airport', 'Airline': 'Delta Airlines, United Airlines, American Airlines', 'AgentName': 'Aiden', 'keyPhrases': 'travel plans, flight options, hotel options, budget, amenities, booking assistance, airport shuttle, fitness center, pool and spa, complimentary', 'topic': 'Booking Travel Arrangements', 'lang': 'en', 'translated_text': "Thank you for calling ABC Travel, my name is Aiden, how can I ass

{'summary': 'John Smith called to get assistance with planning a trip from Atlanta to New York City, selecting airline and hotel within a $1,000 budget. Agent Mia provided multiple options, with John ultimately choosing Delta Airlines and Marriott Marquis.', 'satisfied': 'Yes', 'avgSentiment': 'Positive', 'OriginCity': 'Atlanta', 'DestinationCity': 'New York City', 'Complaint': '', 'Compliment': 'Helpful booking assistance', 'Hotel': 'Marriott Marquis', 'Airline': 'Delta', 'AgentName': 'Mia Robinson', 'keyPhrases': 'travel plans, Atlanta to New York City, airline and hotel options, budget, Delta or American Airlines, Marriott Marquis in Times Square, United Airlines and JetBlue, Hilton Garden Inn Times Square, Hyatt Place New York/Midtown-South, booking', 'topic': 'Travel Planning Assistance', 'lang': 'en', 'translated_text': "Thank you for calling Travel Agency, this is Mia Robinson. How may I assist you today?Hi Mia, my name is John Smith. I'm calling because I'm having some trouble 

{'summary': "Emily Johnson encountered a problem due to her flight to New York City being delayed, which impacted her hotel check-in time. Agent Charlotte Martinez offered to assist by attempting to reschedule the hotel or find other solutions for Emily's urgent travel needs.", 'satisfied': 'No', 'avgSentiment': 'Negative', 'OriginCity': 'Miami', 'DestinationCity': 'New York City', 'Complaint': 'Flight delay affects hotel check-in', 'Compliment': '', 'Hotel': '', 'Airline': 'American Airlines', 'AgentName': 'Charlotte Martinez', 'keyPhrases': 'flight delayed, hotel check-in, urgent travel needs, reschedule hotel, important meeting, American Airlines, Miami to New York, no other flights, trip ruined, seeking solution', 'topic': 'Travel Disruption Assistance', 'lang': 'en', 'translated_text': "Thank you for calling the travel agency, my name is Charlotte Martinez. How may I assist you today? Yes, hi. My name is Emily Johnson and I have a problem with my upcoming trip to New York City. My

SynapseWidget(Synapse.DataFrame, ea8f0734-a5d4-4b21-814a-8da9a8fb01a7)

In [ ]:
# ### THIS BLOCK IS FOR DEBUGGING

# import os
# import openai
# import json
# import time
# import ast

# # Function to get details from a conversation
# def get_details(input_text):
#     time.sleep(4)

#     openai.api_type = openai_api_type
#     openai.api_version = openai_api_version
#     openai.api_base = openai_api_base
#     openai.api_key =  openai_api_key

#    But this

       
#     response = openai.ChatCompletion.create(
#         engine= "gpt-35-turbo-16k", 
#         messages=[
#             {"role": "system", "content": prompt},
#             {"role": "user", "content": input_text}
#         ]
#     )

#     print(response['choices'][0]['message']['content'])

#     # Parse the response from the API
#     result = ast.literal_eval(response['choices'][0]['message']['content'])
#     # If 'summary' is found in the result, print and return the result

#     print(result)
#     return result


# # input_str = '''Thank you for reaching out to the travel agency contact center. My name is Sarah Thompson. How may I assist you today?Hi Sarah, my name is Lisa Johnson. I recently traveled from Chicago to London and I had a terrible experience with the airline and hotel. I'm really frustrated with the service I received.I'm sorry to hear that, Lisa. Can you please provide me with some details about your trip so I can better understand the situation?Sure. I flew with United Airlines from Chicago to London, and I stayed at the Park Plaza Westminster Bridge hotel in London. I encountered issues with both during my trip.I apologize for any inconvenience you experienced. Could you please explain the specific problems you encountered with United Airlines?Absolutely. Firstly, the flight was delayed for more than three hours without any proper explanation. This caused a lot of inconvenience as I had connecting flights and had to reschedule my entire itinerary. Secondly, the onboard service was subpar. The flight attendants seemed disinterested and were not attentive to the passengers' needs.I understand how frustrating these situations can be, Lisa. I apologize for the lack of communication and the inconvenience caused by the delay. Delayed flights can be quite disruptive. Regarding the onboard service, I apologize for the unprofessional behavior of the flight attendants. I will make a note of your concerns and forward them to the airline for review.Thank you, Sarah. I appreciate your understanding. Now, regarding my hotel stay at the Park Plaza Westminster Bridge, the room was not up to standard. It was not properly cleaned, and there were maintenance issues with the bathroom.I apologize for the hotel's shortcomings, Lisa. It can be disappointing when accommodations don't meet expectations. I will contact the hotel management to address the cleanliness and maintenance issues you faced. In the meantime, is there anything specific you would like me to convey to the hotel?I would like them to know that I expect better cleanliness and maintenance in their rooms. It was really disappointing, especially considering the hotel's reputation.I completely understand, Lisa. I will communicate your concerns to the hotel management and emphasize the importance of ensuring a high level of cleanliness and maintenance throughout their property.Thank you, Sarah. I appreciate your assistance. Is there anything else you can do to help resolve these issues?Absolutely, Lisa. To further assist you, I will contact United Airlines to see if they can offer any compensation for the delay and address your concerns about the onboard service. Additionally, I will follow up with the Park Plaza Westminster Bridge hotel to ensure they take appropriate action regarding the cleanliness and maintenance issues in your room. I will keep you updated throughout the process.That sounds good, Sarah. I appreciate your efforts in resolving these matters. Thank you for your assistance.You're most welcome, Lisa. It is our priority to ensure that our customers have a pleasant travel experience. I will work diligently to resolve these issues for you. If you have any other questions or concerns, please don't hesitate to contact me.Thank you, Sarah. I will definitely reach out if I need any further assistance.'''
# # res = get_details(input_str)

# from pyspark.sql import Row

# from pyspark.sql.types import *
# from pyspark.sql.functions import *

# schema = StructType([
#              StructField("summary", StringType(), True),
#              StructField("satisfied", StringType(), True),
#              StructField("avgSentiment", StringType(), True),
#              StructField("OriginCity", StringType(), True),
#              StructField("DestinationCity", StringType(), True),
#              StructField("Complaint", StringType(), True),
#              StructField("Compliment", StringType(), True),
#              StructField("Hotel", StringType(), True),
#              StructField("Airline", StringType(), True),
#              StructField("AgentName", StringType(), True),
#              StructField("keyPhrases", StringType(), True),
#              StructField("topic", StringType(), True),
#              StructField("lang", StringType(), True),
#              StructField("translated_text", StringType(), True)
#          ])

# get_detail_udf = udf(lambda content: get_details(content),returnType=schema)

# df_processed = selected_df.select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason",
#     "CallerID","Merged_content","Merged_content_agent","Merged_content_user"]) \
#                 .withColumn("Details", get_detail_udf(col("Merged_content"))) \
#                 .select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason","CallerID","Merged_content","Merged_content_agent","Merged_content_user", \
#                           col("Details.summary").alias("summary"), \
#                           col("Details.satisfied").alias("satisfied"), \
#                           col("Details.avgSentiment").alias("avgSentiment"), \
#                           col("Details.OriginCity").alias("OriginCity"), \
#                           col("Details.DestinationCity").alias("DestinationCity"), \
#                           col("Details.Complaint").alias("Complaint"), \
#                           col("Details.Compliment").alias("Compliment"), \
#                           col("Details.Hotel").alias("Hotel"), \
#                           col("Details.Airline").alias("Airline"), \
#                           # col("Details.AgentName").alias("AgentName"), \
#                           col("Details.keyPhrases").alias("keyPhrases"), \
#                           col("Details.topic").alias("topic"), \
#                           col("Details.lang").alias("lang"), \
#                           col("Details.translated_text").alias("translated_text")]) 
# display(df_processed)

StatementMeta(, , , Waiting, )

In [ ]:
# #THIS BLOCK IS FOR DEBUGGING 
# # Importing necessary libraries
# import os
# import openai
# import json
# import time
# import ast

# # Function to get details from a conversation
# def get_details(input_text):
#     # Pause execution for 4 seconds
#     time.sleep(4)

#     # Set OpenAI API parameters
#     openai.api_type = openai_api_type
#     openai.api_version = openai_api_version
#     openai.api_base = openai_api_base
#     openai.api_key =  openai_api_key

#     # Construct the prompt for the OpenAI API
#     prompt = '''You are a JSON formatter for extracting information out of a single chat conversation. 
#             Summarize the conversation, key: summary .
#             Is the customer satisfied with the agent interaction (Yes or No), key: satisfied . 
#             Identify the sentiment of the customer as (Positive, Negative or Neutral),key : avgSentiment . 
#             Identify the origin city of travel,key: OriginCity . 
#             Identify the destination city of travel,key : DestinationCity . 
#             Identify the single primary complaint of the conversation in 6 words or less,key: Complaint . 
#             Identify the single primary compliment of the conversation in 6 words or less,key: Compliment . 
#             Identify the name of hotel that was mentioned,key: Hotel . 
#             Identify the name of airline if mentioned,key: Airline . 
#             Identify the name of the agent,key: AgentName .
#             Identify the top 10 key phrases as comma seperated string excluding people names , key: keyPhrases .
#             Identify the main topic, key: topic .
#             Identify the language of the text using ISO 639 two letter language identifier, key: lang .
#             Identify input_text translated to english, return the same text if already in english, key: translated_text .
#             Answer in JSON machine-readable format, using the keys from above. 
#             Pretty print the JSON and make sure that it is properly closed at the end and do not generate any other content.'''

#     # Call OpenAI API with the constructed prompt
#     response = openai.ChatCompletion.create(
#         engine= "gpt-35-turbo-16k", 
#         messages=[
#             {"role": "system", "content": prompt},
#             {"role": "user", "content": input_text}
#         ]
#     )

#     # Return the response content directly
#     return response['choices'][0]['message']['content']

# # Import necessary PySpark functions and types
# from pyspark.sql import Row
# from pyspark.sql.types import *
# from pyspark.sql.functions import *

# # Define the schema for the DataFrame
# schema = StructType([
#              StructField("summary", StringType(), True),
#              StructField("satisfied", StringType(), True),
#              StructField("avgSentiment", StringType(), True),
#              StructField("OriginCity", StringType(), True),
#              StructField("DestinationCity", StringType(), True),
#              StructField("Complaint", StringType(), True),
#              StructField("Compliment", StringType(), True),
#              StructField("Hotel", StringType(), True),
#              StructField("Airline", StringType(), True),
#              StructField("AgentName", StringType(), True),
#              StructField("keyPhrases", StringType(), True),
#              StructField("topic", StringType(), True),
#              StructField("lang", StringType(), True),
#              StructField("translated_text", StringType(), True)
#          ])

# # Define a user-defined function (UDF) that applies the get_details function to a column
# get_detail_udf = udf(lambda content: get_details(content),returnType=schema)

# # Apply the UDF to the DataFrame and select the necessary columns
# df_processed = selected_df.select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason",
#     "CallerID","Merged_content","Merged_content_agent","Merged_content_user"]) \
#                 .withColumn("Details", get_detail_udf(col("Merged_content"))) \
#                 .select(["ConversationId", "ConversationDate", "EndTime","StartTime","Duration","AgentId","AgentName","Team","ResolutionStatus","CallReason","CallerID","Merged_content","Merged_content_agent","Merged_content_user", \
#                           col("Details.summary").alias("summary"), \
#                           col("Details.satisfied").alias("satisfied"), \
#                           col("Details.avgSentiment").alias("avgSentiment"), \
#                           col("Details.OriginCity").alias("OriginCity"), \
#                           col("Details.DestinationCity").alias("DestinationCity"), \
#                           col("Details.Complaint").alias("Complaint"), \
#                           col("Details.Compliment").alias("Compliment"), \
#                           col("Details.Hotel").alias("Hotel"), \
#                           col("Details.Airline").alias("Airline"), \
#                           # col("Details.AgentName").alias("AgentName"), \
#                           col("Details.keyPhrases").alias("keyPhrases"), \
#                           col("Details.topic").alias("topic"), \
#                           col("Details.lang").alias("lang"), \
#                           col("Details.translated_text").alias("translated_text")]) 

# # Display the processed DataFrame
# df_processed.show()


In [13]:
# Save processed records to ckm_conv_processed_raw table
df_processed.write.format('delta').mode('append').option("overwriteSchema", "true").saveAsTable('ckm_conv_processed_raw')

StatementMeta(, , , Waiting, )

In [ ]:
table_name = "ckm_conv_processed_raw"  # Replace with your table name

# Get the schema of the table
table_schema = spark.table(table_name).schema

# Compare the schema of DataFrame and the table
if str(df_processed.schema) == str(table_schema):
    print("The DataFrame schema matches the table schema.")
else:
    print("The DataFrame schema does not match the table schema.")
    print("\nDataFrame schema:\n" + str(df_processed.schema))
    print("\nTable schema:\n" + str(table_schema))

In [ ]:
# df = spark.sql("SELECT * FROM ckm_conv_processed_raw LIMIT 1000")
# display(df)

In [ ]:
# df = spark.sql("SELECT ConversationId,AgentId,CallerID,avgSentiment,lang,summary  FROM ckm_conv_processed_raw LIMIT 1000")
# display(df)